# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
import geoviews as gv
import geoviews.tile_sources as gts
import holoviews as hv
from holoviews.plotting.bokeh import BokehRenderer

# Import API key
from apis_key import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,manono,-7.3000,27.4167,21.16,93,33,1.96,CD,1681757630
1,1,rio grande,-32.0350,-52.0986,21.06,69,68,6.83,BR,1681757631
2,2,carnarvon,-24.8667,113.6333,21.98,40,49,5.94,AU,1681757631
3,3,pacific grove,36.6177,-121.9166,15.21,68,0,5.66,US,1681757519
4,4,merauke,-8.4667,140.3333,26.85,82,100,1.94,ID,1681757632


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
city_data_df = city_data_df.rename(columns={
    'Lat' : 'latitude',
    'Lng' : 'longitude'
})

city_data_map = city_data_df.hvplot.points('longitude'
                                           , 'latitude'
                                           , geo = True
                                           , tiles = 'OSM'
                                           , size = 'Humidity'
                                           , color = 'City'
                                           , alpha = 0.5
                                           , frame_width = 800
                                           ,frame_height = 600
                                          )

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
clean_city_data = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]
clean_city_data

,City_ID,City,latitude,longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,port alfred,-33.5906,26.8910,21.16,55,0,1.24,ZA,1681757637
49,49,saint george,37.1041,-113.5841,24.13,21,0,1.54,US,1681757425
59,59,east london,-33.0153,27.9116,21.52,64,0,3.09,ZA,1681757660
144,144,kahului,20.8947,-156.4700,26.58,72,0,0.89,US,1681757656
148,148,victoria,22.2855,114.1577,26.29,88,0,2.68,HK,1681757569
166,166,makakilo city,21.3469,-158.0858,26.18,73,0,2.06,US,1681757708
235,235,gat,31.6100,34.7642,25.22,62,0,1.28,IL,1681757745
304,304,hambantota,6.1241,81.1185,26.19,100,0,0.00,LK,1681757776
322,322,monte carmelo,-18.7247,-47.4986,26.42,61,0,2.72,BR,1681757787
390,390,safaga,26.7292,33.9365,25.71,41,0,0.82,EG,1681757816


In [13]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_data = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
clean_city_data = clean_city_data.dropna()

# Display sample data
clean_city_data

,City_ID,City,latitude,longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,port alfred,-33.5906,26.8910,21.16,55,0,1.24,ZA,1681757637
49,49,saint george,37.1041,-113.5841,24.13,21,0,1.54,US,1681757425
59,59,east london,-33.0153,27.9116,21.52,64,0,3.09,ZA,1681757660
144,144,kahului,20.8947,-156.4700,26.58,72,0,0.89,US,1681757656
148,148,victoria,22.2855,114.1577,26.29,88,0,2.68,HK,1681757569
166,166,makakilo city,21.3469,-158.0858,26.18,73,0,2.06,US,1681757708
235,235,gat,31.6100,34.7642,25.22,62,0,1.28,IL,1681757745
304,304,hambantota,6.1241,81.1185,26.19,100,0,0.00,LK,1681757776
322,322,monte carmelo,-18.7247,-47.4986,26.42,61,0,2.72,BR,1681757787
390,390,safaga,26.7292,33.9365,25.71,41,0,0.82,EG,1681757816


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_data[['City', 'Country', 'latitude', 'longitude', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,latitude,longitude,Humidity,Hotel Name
19,port alfred,ZA,-33.5906,26.8910,55,
49,saint george,US,37.1041,-113.5841,21,
59,east london,ZA,-33.0153,27.9116,64,
144,kahului,US,20.8947,-156.4700,72,
148,victoria,HK,22.2855,114.1577,88,
166,makakilo city,US,21.3469,-158.0858,73,
235,gat,IL,31.6100,34.7642,62,
304,hambantota,LK,6.1241,81.1185,100,
322,monte carmelo,BR,-18.7247,-47.4986,61,
390,safaga,EG,26.7292,33.9365,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories' : 'accommodation.hotel',
          'limit' : 1,
          'apiKey' : geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude =  row['latitude']
    longitude = row['longitude']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port alfred - nearest hotel: No hotel found
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
east london - nearest hotel: No hotel found
kahului - nearest hotel: Maui Seaside Hotel
victoria - nearest hotel: 四季酒店 Four Seasons Hotel
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
gat - nearest hotel: No hotel found
hambantota - nearest hotel: Oasis Aurweda Resort Hambathota
monte carmelo - nearest hotel: Hotel Dona Wanda
safaga - nearest hotel: Dolphin Hotel
luang prabang - nearest hotel: Amantaka
reconquista - nearest hotel: Hotel San Martin
doha - nearest hotel: Al MIrqab


,City,Country,latitude,longitude,Humidity,Hotel Name
19,port alfred,ZA,-33.5906,26.8910,55,No hotel found
49,saint george,US,37.1041,-113.5841,21,"The Advenire, an Autograph Collection Hotel"
59,east london,ZA,-33.0153,27.9116,64,No hotel found
144,kahului,US,20.8947,-156.4700,72,Maui Seaside Hotel
148,victoria,HK,22.2855,114.1577,88,四季酒店 Four Seasons Hotel
166,makakilo city,US,21.3469,-158.0858,73,Embassy Suites by Hilton Oahu Kapolei
235,gat,IL,31.6100,34.7642,62,No hotel found
304,hambantota,LK,6.1241,81.1185,100,Oasis Aurweda Resort Hambathota
322,monte carmelo,BR,-18.7247,-47.4986,61,Hotel Dona Wanda
390,safaga,EG,26.7292,33.9365,41,Dolphin Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(       'longitude'
                                           , 'latitude'
                                           , geo = True
                                           , tiles = 'OSM'
                                           , size = 'Humidity'
                                           , color = 'City'
                                           , alpha = 0.5
                                           , hover_cols='all'
                                           , frame_width = 800
                                           ,frame_height = 600
                                          )

# Display the map
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity,index,Country,Hotel Name)